In [377]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np


In [378]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(device)

cpu


In [379]:
# Creating sliding windows
def slide_windows(features):

    # We need to ensure we have data for t-1, t, t+1 without index errors
    windowed_data = []
    predict_prices = []  # List to store the target 'Close' prices
    cycle_data=[]

    # Handling daily cycles
    cycle_length = 7
    for cycle in range(len(features) - cycle_length + 1):
        start_index = cycle
        end_index = start_index + cycle_length
        cycle_datas = features.iloc[start_index:end_index,1:6].values
        cycle_data.append(cycle_datas)
        
    # Creating sliding windows within the cycle
    for i in range(14,len(cycle_data)-7):  # Avoiding index error by stopping before the last day
        pre_previous = cycle_data[i - 14]
        previous = cycle_data[i- 7]
        current_state = cycle_data[i]

        combined_features = np.concatenate([pre_previous, previous, current_state]).reshape(1,-1).squeeze()
        windowed_data.append(combined_features)
        predict_prices.append(cycle_data[i+7][-1][3])
    
    return windowed_data, predict_prices


In [380]:
# Load data
row_data = pd.read_csv('TSLA_stock_data_2023.csv')

windowed_data, predict_prices = slide_windows(row_data)
# Convert to PyTorch tensors
state = torch.tensor(windowed_data, dtype=torch.float32).to(device)
predict_price = torch.tensor(predict_prices, dtype=torch.float32).to(device)
# state[2]

In [381]:
# divide the data into training part and test part
state_train, state_test, predict_price_train,predict_price_test= train_test_split(state, predict_price,test_size=0.2, random_state=42)
train_loader = DataLoader(TensorDataset(state_train, predict_price_train), batch_size=200, shuffle=True)
test_loader = DataLoader(TensorDataset(state_test, predict_price_test), batch_size=300)


In [382]:
# construct NN
class NN(nn.Module):
    def __init__(self, n_observations):
        super(NN, self).__init__()
        self.layer1 = nn.Linear(n_observations, 128)
        self.layer2 = nn.Linear(128, 128)
        self.layer3 = nn.Linear(128, 1)

    def forward(self, x):
        x = F.relu(self.layer1(x))
        x = F.relu(self.layer2(x))
        return self.layer3(x)
    

In [383]:
# Optimazation function, it do one step of gredient decent, 
def optimize_model():
    for state, target in train_loader:
        current_value=value_net(state).squeeze()              
        criterion = nn.SmoothL1Loss()
        loss=criterion(current_value,target)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    for state, target in test_loader:
        with torch.no_grad():
            test_valur=value_net(state).squeeze()                 
            l_test=criterion(test_valur,target)
    return loss.item(),l_test.item()

In [385]:
learning_rates = [1e-10, 1e-5, 1e-3]  # Possible learning rates
weight_decays = [1e-2, 1e-3, 1e-4, 1e-5]  # Possible weight decay values
n_observations = len(state[0])
epochs=2000

best_lr = None
best_weight_decay = None
lowest_test_loss = float('inf')  # Initialize with infinity to ensure any first result is better

# value_net is your prediction function, take state as input and output is the prediction price
value_net = NN(n_observations).to(device)

for LR in learning_rates:
    for wd in weight_decays:
        print(f"Testing with LR = {LR} and Weight Decay = {wd}")
        # 'optimize'  is an easy package to do Gredient decent, 'Adam' is a method to let learing rate decay as step go.
        optimizer = optim.Adam(value_net.parameters(), lr=LR,weight_decay=wd)
    
        # Run the training loop for this combination of parameters
        for epoch in range(epochs):
            l_train, l_test = optimize_model()  # Use the current lr and wd in your optimization
            if epoch % 100 == 0:  # Report every 100 epochs
                print(f'Epoch [{epoch+1}/{epochs}], L_train: {l_train}, L_test: {l_test}')

            # Save parameters if this is the best we've seen
            if l_test < lowest_test_loss:
                lowest_test_loss = l_test
                best_lr = LR
                best_weight_decay = wd

# Output the best parameters and the test loss achieved with them
print(f"Best Learning Rate: {best_lr}, Best Weight Decay: {best_weight_decay}, Lowest L_test: {lowest_test_loss}")
#

Testing with LR = 1e-10 and Weight Decay = 0.01
Epoch [1/2000], L_train: 205730.203125, L_test: 258230.234375
Epoch [101/2000], L_train: 236961.34375, L_test: 258228.34375
Epoch [201/2000], L_train: 203048.046875, L_test: 258226.484375
Epoch [301/2000], L_train: 214734.578125, L_test: 258224.671875
Epoch [401/2000], L_train: 240570.296875, L_test: 258222.796875
Epoch [501/2000], L_train: 210176.984375, L_test: 258220.984375
Epoch [601/2000], L_train: 235544.90625, L_test: 258219.09375
Epoch [701/2000], L_train: 225890.140625, L_test: 258217.203125
Epoch [801/2000], L_train: 223120.859375, L_test: 258215.390625
Epoch [901/2000], L_train: 230204.515625, L_test: 258213.53125
Epoch [1001/2000], L_train: 227778.609375, L_test: 258211.75
Epoch [1101/2000], L_train: 193577.9375, L_test: 258209.890625
Epoch [1201/2000], L_train: 220896.53125, L_test: 258208.0625
Epoch [1301/2000], L_train: 230291.53125, L_test: 258206.234375
Epoch [1401/2000], L_train: 220352.09375, L_test: 258204.421875
Epoch

In [395]:
# continue optimize
# start from last training result
epochs=10000
LR = 1e-3
Weight_decay=0.01
value_net.train()
for epoch in range(epochs):
    l_train,l_test=optimize_model()
    if epoch % 10 ==0:
        print(f'Epoch [{epoch+1}/{epochs}], L_train: {l_train},L_test: {l_test}')

Epoch [1/10000], L_train: 34.40766906738281,L_test: 41.59270477294922
Epoch [101/10000], L_train: 36.63518524169922,L_test: 41.575870513916016
Epoch [201/10000], L_train: 33.14952087402344,L_test: 41.59728240966797
Epoch [301/10000], L_train: 35.45770263671875,L_test: 41.57453536987305
Epoch [401/10000], L_train: 32.29743194580078,L_test: 40.998802185058594
Epoch [501/10000], L_train: 34.25470733642578,L_test: 41.23904800415039
Epoch [601/10000], L_train: 80.77875518798828,L_test: 72.77326965332031
Epoch [701/10000], L_train: 44.58434295654297,L_test: 51.333740234375
Epoch [801/10000], L_train: 43.650020599365234,L_test: 54.22959518432617
Epoch [901/10000], L_train: 49.00927734375,L_test: 53.5388069152832
Epoch [1001/10000], L_train: 40.38454055786133,L_test: 54.02301025390625
Epoch [1101/10000], L_train: 41.03861618041992,L_test: 53.37639617919922
Epoch [1201/10000], L_train: 41.92079162597656,L_test: 49.35393524169922
Epoch [1301/10000], L_train: 44.359127044677734,L_test: 43.9361724

In [ ]:
#save last line results
torch.save(value_net.state_dict(),'model_parameters.pth')

In [ ]:
#load results
loaded_model = NN(n_observations).to(device)
value_net.load_state_dict(torch.load('model_parameters.pth'))

In [ ]:
# Load data
windowed_data, predict_prices = slide_windows(pd.read_csv('TSLA_stock_data_2024.csv'))
# Convert to PyTorch tensors
state_new = torch.tensor(windowed_data, dtype=torch.float32).to(device)
real_price = torch.tensor(predict_prices, dtype=torch.float32).to(device)

#load the trained parameter to the model
# loaded_model = NN(n_observations).to(device)
# loaded_model.load_state_dict(torch.load('model_parameters.pth')) 
# value_net=loaded_model 

# predict your own price
value_net.eval()  
with torch.no_grad():
    criterion = nn.SmoothL1Loss()
    predict_price=loaded_model(state_new).squeeze() #predict price
    l_test=criterion(real_price,predict_price) #calculate difference
print(l_test)

RuntimeError: Attempting to deserialize object on a CUDA device but torch.cuda.is_available() is False. If you are running on a CPU-only machine, please use torch.load with map_location=torch.device('cpu') to map your storages to the CPU.